<a href="https://colab.research.google.com/github/DeepakRautella/google_collab/blob/main/keras_tuner(hyper_tune).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df=pd.read_csv('/content/diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
df.corrwith(df['Outcome'])

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
dtype: float64

In [6]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [8]:
X=scaler.fit_transform(X)

In [9]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [10]:
X.shape

(768, 8)

In [11]:
y.shape

(768,)

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [14]:
model=Sequential()

model.add(Dense(32,input_dim=8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])


In [15]:
model.fit(X_train,Y_train,validation_split=0.2,batch_size=32,epochs=100)

Epoch 1/100
16/16 [==============================] - 1s 26ms/step - loss: 0.6263 - accuracy: 0.6375 - val_loss: 0.5869 - val_accuracy: 0.7317
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5844 - accuracy: 0.6884 - val_loss: 0.5624 - val_accuracy: 0.7073
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5547 - accuracy: 0.7332 - val_loss: 0.5427 - val_accuracy: 0.7236
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5330 - accuracy: 0.7556 - val_loss: 0.5297 - val_accuracy: 0.7317
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5164 - accuracy: 0.7637 - val_loss: 0.5198 - val_accuracy: 0.7398
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5037 - accuracy: 0.7739 - val_loss: 0.5111 - val_accuracy: 0.7480
Epoch 7/100
16/16 [==============================] - 0s 8ms/step - loss: 0.4936 - accuracy: 0.7760 - val_loss: 0.5051 - val_accuracy: 0.7480
Epoch 8/100


**Keras Tuner**

after starting model sequential we are lost in models variables selection what to choose, which activation function , which optimizer, how many epoches, number of neurons and how deep a neural network needed for get best model out of our dataset.

to solve this issue developer comes up with keras tuner which works same as hypertunning in machine learning ex(grid search cv ,random search cv)

In [16]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


**Hyper tunning for optimizer**
first build a function

In [108]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(32,input_dim=8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  
  return model

In [109]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy', max_trials=5,directory='s')


In [110]:
tuner.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 07s


In [111]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [112]:
model_=tuner.get_best_models(num_models=1)[0]

In [113]:
model_.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [114]:
model_.fit(X_train,Y_train,batch_size=32,initial_epoch=6,epochs=100,validation_data=(X_test,Y_test))

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.4865 - accuracy: 0.7606 - val_loss: 0.5227 - val_accuracy: 0.7532
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4801 - accuracy: 0.7573 - val_loss: 0.5201 - val_accuracy: 0.7468
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4738 - accuracy: 0.7638 - val_loss: 0.5154 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4694 - accuracy: 0.7687 - val_loss: 0.5140 - val_accuracy: 0.7662
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4647 - accuracy: 0.7704 - val_loss: 0.5126 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4607 - accuracy: 0.7720 - val_loss: 0.5124 - val_accuracy: 0.7597
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4573 - accuracy: 0.7736 - val_loss: 0.5115 - val_accuracy: 0.7727
Epoch 14

**hyper tune**
Number of neuron in a single layer

In [115]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('node',8,128,step=8)
  model.add(Dense(units=units,input_dim=8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  
  return model

In [116]:
tuner_=kt.RandomSearch(build_model,objective='val_accuracy', max_trials=5,directory='sa')

In [117]:
tuner_.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 11s


In [118]:
tuner_.get_best_hyperparameters()[0].values

{'node': 120, 'optimizer': 'rmsprop'}

In [119]:
model_1=tuner_.get_best_models(num_models=1)[0]

In [120]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               1080      
                                                                 
 dense_1 (Dense)             (None, 1)                 121       
                                                                 
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________


In [121]:
model_1.fit(X_train,Y_train,batch_size=32,initial_epoch=10,epochs=100,validation_data=(X_test,Y_test))

Epoch 11/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5074 - accuracy: 0.7622 - val_loss: 0.5144 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4836 - accuracy: 0.7687 - val_loss: 0.5033 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4704 - accuracy: 0.7687 - val_loss: 0.5017 - val_accuracy: 0.7857
Epoch 14/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4608 - accuracy: 0.7671 - val_loss: 0.5020 - val_accuracy: 0.7662
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4539 - accuracy: 0.7752 - val_loss: 0.5031 - val_accuracy: 0.7532
Epoch 16/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4492 - accuracy: 0.7801 - val_loss: 0.5052 - val_accuracy: 0.7403
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4457 - accuracy: 0.7834 - val_loss: 0.5066 - val_accuracy: 0.7403
Epoch

**hyper tune activation function**

In [125]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('node',8,128)
  model.add(Dense(units=units,input_dim=8,activation=hp.Choice("activation", ["relu", "tanh"])))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  
  return model


In [126]:
tuner_1=kt.RandomSearch(build_model,objective='val_accuracy', max_trials=5,directory='sb')

In [127]:
tuner_1.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 09s


In [128]:
tuner_1.get_best_hyperparameters()[0].values

{'node': 65, 'activation': 'relu', 'optimizer': 'adam'}

In [129]:
model_3=tuner_1.get_best_models(num_models=1)[0]

In [130]:
model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 65)                585       
                                                                 
 dense_1 (Dense)             (None, 1)                 66        
                                                                 
Total params: 651
Trainable params: 651
Non-trainable params: 0
_________________________________________________________________


In [131]:
# to check which activation on a particular layers
layer=model_3.get_layer('dense')

In [132]:

layer.activation

<function keras.activations.relu(x, alpha=0.0, max_value=None, threshold=0.0)>

In [139]:
model_3.fit(X_train,Y_train,batch_size=32,initial_epoch=10,epochs=100,validation_data=(X_test,Y_test))

Epoch 11/100
20/20 [==============================] - 1s 11ms/step - loss: 0.4807 - accuracy: 0.7736 - val_loss: 0.4962 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4706 - accuracy: 0.7752 - val_loss: 0.4932 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4624 - accuracy: 0.7752 - val_loss: 0.4901 - val_accuracy: 0.7857
Epoch 14/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4568 - accuracy: 0.7785 - val_loss: 0.4891 - val_accuracy: 0.7857
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4518 - accuracy: 0.7818 - val_loss: 0.4897 - val_accuracy: 0.7857
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4483 - accuracy: 0.7850 - val_loss: 0.4921 - val_accuracy: 0.7792
Epoch 17/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4455 - accuracy: 0.7834 - val_loss: 0.4939 - val_accuracy: 0.7792
Epoch

**hyper tune for number of hidden layers**


In [133]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('node',8,128)
  activation=hp.Choice("activation", ["relu", "tanh"]),
  model.add(Dense(units=units,input_dim=8,activation=hp.Choice("activation", ["relu", "tanh"])))
  
  for i in range(hp.Int('num_of_layers',min_value=1,max_value=10)):
    units=hp.Int('node',8,128)
    activation=hp.Choice("activation", ["relu", "tanh"]),
    model.add(Dense(units=units,input_dim=8,activation=hp.Choice("activation", ["relu", "tanh"])))
  
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  
  return model


In [134]:
tuner_2=kt.RandomSearch(build_model,objective='val_accuracy', max_trials=5,directory='sc')


In [135]:
tuner_2.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 18s


In [136]:
tuner_2.get_best_hyperparameters()[0].values

{'node': 121, 'activation': 'tanh', 'num_of_layers': 4, 'optimizer': 'rmsprop'}

In [137]:
model_best=tuner_2.get_best_models(num_models=1)[0]

In [140]:
model_best.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 121)               1089      
                                                                 
 dense_1 (Dense)             (None, 121)               14762     
                                                                 
 dense_2 (Dense)             (None, 121)               14762     
                                                                 
 dense_3 (Dense)             (None, 121)               14762     
                                                                 
 dense_4 (Dense)             (None, 121)               14762     
                                                                 
 dense_5 (Dense)             (None, 1)                 122       
                                                                 
Total params: 60,259
Trainable params: 60,259
Non-traina

In [138]:
model_best.fit(X_train,Y_train,batch_size=32,initial_epoch=10,epochs=100,validation_data=(X_test,Y_test))

Epoch 11/100
20/20 [==============================] - 1s 13ms/step - loss: 0.4722 - accuracy: 0.7720 - val_loss: 0.5677 - val_accuracy: 0.7208
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4704 - accuracy: 0.7769 - val_loss: 0.5302 - val_accuracy: 0.7468
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4621 - accuracy: 0.7720 - val_loss: 0.4826 - val_accuracy: 0.7792
Epoch 14/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4628 - accuracy: 0.7671 - val_loss: 0.6133 - val_accuracy: 0.7078
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4699 - accuracy: 0.7622 - val_loss: 0.4813 - val_accuracy: 0.7727
Epoch 16/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4462 - accuracy: 0.7834 - val_loss: 0.5639 - val_accuracy: 0.6883
Epoch 17/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4520 - accuracy: 0.7769 - val_loss: 0.5005 - val_accuracy: 0.7727
Epoch

**too much overfitting**
add drop out layer

**learning rate in optimizer **

In [145]:
from keras.layers import Dropout

In [159]:

def build_model(hp):
  model=Sequential()
  units=hp.Int('node',8,128)
  activation=hp.Choice("activation", ["relu", "tanh"]),
  model.add(Dense(units=units,input_dim=8,activation=hp.Choice("activation", ["relu", "tanh"])))
  
  for i in range(hp.Int('num_of_layers',min_value=1,max_value=10)):
    units=hp.Int('node',8,128)
    activation=hp.Choice("activation", ["relu", "tanh"]),
    model.add(Dense(units=units,input_dim=8,activation=hp.Choice("activation", ["relu", "tanh"])))
    model.add(Dropout(0.25))


  model.add(Dense(1,activation='sigmoid'))
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"]
        )
  
  return model


In [160]:
tuner_3=kt.RandomSearch(build_model,objective='val_accuracy', max_trials=5,directory='sr')


In [161]:
tuner_3.search(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 16s


In [162]:
tuner_3.get_best_hyperparameters()[0].values

{'node': 38,
 'activation': 'relu',
 'num_of_layers': 7,
 'lr': 0.0021599950963650445}

In [163]:
model_best_=tuner_3.get_best_models(num_models=1)[0]

In [164]:
model_best_.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 38)                342       
                                                                 
 dense_1 (Dense)             (None, 38)                1482      
                                                                 
 dropout (Dropout)           (None, 38)                0         
                                                                 
 dense_2 (Dense)             (None, 38)                1482      
                                                                 
 dropout_1 (Dropout)         (None, 38)                0         
                                                                 
 dense_3 (Dense)             (None, 38)                1482      
                                                                 
 dropout_2 (Dropout)         (None, 38)                0

In [165]:
model_best_.fit(X_train,Y_train,batch_size=32,initial_epoch=10,epochs=100,validation_data=(X_test,Y_test))

Epoch 11/100
20/20 [==============================] - 2s 18ms/step - loss: 0.4742 - accuracy: 0.7590 - val_loss: 0.5688 - val_accuracy: 0.7338
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4610 - accuracy: 0.7720 - val_loss: 0.5443 - val_accuracy: 0.6883
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4735 - accuracy: 0.7720 - val_loss: 0.5269 - val_accuracy: 0.7208
Epoch 14/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4558 - accuracy: 0.7883 - val_loss: 0.5331 - val_accuracy: 0.7208
Epoch 15/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4752 - accuracy: 0.7590 - val_loss: 0.5464 - val_accuracy: 0.7338
Epoch 16/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4729 - accuracy: 0.7818 - val_loss: 0.5331 - val_accuracy: 0.7208
Epoch 17/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4459 - accuracy: 0.7785 - val_loss: 0.5536 - val_accuracy: 0.6818
Epoch

In [166]:
tuner_3.results_summary()

Results summary
Results in sr/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
node: 38
activation: relu
num_of_layers: 7
lr: 0.0021599950963650445
Score: 0.7597402334213257
Trial summary
Hyperparameters:
node: 40
activation: tanh
num_of_layers: 1
lr: 0.00027228398750180077
Score: 0.7532467246055603
Trial summary
Hyperparameters:
node: 71
activation: relu
num_of_layers: 1
lr: 0.00016018560667549093
Score: 0.7207792401313782
Trial summary
Hyperparameters:
node: 47
activation: relu
num_of_layers: 7
lr: 0.0001964530109548961
Score: 0.6428571343421936
Trial summary
Hyperparameters:
node: 33
activation: relu
num_of_layers: 10
lr: 0.0005170567445148574
Score: 0.6428571343421936
